In [14]:
import torch

class ContinuousBayesianEstimator:
    def __init__(self, prior_mean=0.0, prior_var=1.0, prior_dof=1.0, prior_scale=1.0):
        """
        Initializes the Bayesian estimator with Normal-Inverse-Gamma priors.
        
        Args:
            prior_mean (float): Prior mean of the Gaussian.
            prior_var (float): Prior variance.
            prior_dof (float): Degrees of freedom for the Inverse-Gamma prior on variance.
            prior_scale (float): Scale parameter for the Inverse-Gamma prior on variance.
        """
        # Initialize priors
        self.prior_mean = prior_mean
        self.prior_var = prior_var
        self.prior_dof = prior_dof
        self.prior_scale = prior_scale

        # Initialize posterior parameters as priors
        self.posterior_mean = prior_mean
        self.posterior_var = prior_var
        self.posterior_dof = prior_dof
        self.posterior_scale = prior_scale

    def update_posterior(self, data):
        """
        Update the posterior distribution parameters based on observed data.
        
        Args:
            data (Tensor): Observed data (continuous values).
        """
        n = data.shape[0]  # Number of data points
        sample_mean = data.mean()
        sample_var = data.var(unbiased=False)

        # Update posterior parameters
        self.posterior_dof += n
        self.posterior_mean = (self.prior_var * sample_mean + n * self.prior_mean) / (self.prior_var + n)
        self.posterior_var = (
            self.prior_scale
            + 0.5 * n * sample_var
            + (n * self.prior_var * (sample_mean - self.prior_mean) ** 2) / (2 * (self.prior_var + n))
        )
        self.posterior_scale = self.posterior_var / self.posterior_dof

    def sample_posterior(self, n_samples=1):
        """
        Sample from the posterior distribution of the mean using Student's t-distribution.
        
        Args:
            n_samples (int): Number of samples to generate.
        
        Returns:
            Tensor: Samples from the posterior distribution of the mean.
        """
        posterior_var = self.posterior_scale / self.posterior_dof
        posterior_std = torch.sqrt(posterior_var)
        
        # Sample from Student's t-distribution
        student_t_dist = torch.distributions.StudentT(self.posterior_dof, self.posterior_mean, posterior_std)
        return student_t_dist.sample((n_samples,))

    def compute_cpds(self, data, given_values):
        """
        Compute conditional probabilities for continuous values.
        
        Args:
            data (Tensor): Observed data (continuous values).
            given_values (Tensor): Values to condition on.
        
        Returns:
            Tensor: Conditional probability densities.
        """
        epsilon = 1e-6  # Small value to prevent division by zero or negative variances
        
        # Calculate joint statistics
        joint_data = torch.cat((data, given_values))
        joint_mean = joint_data.mean()
        joint_var = joint_data.var(unbiased=False)
        
        # Set conditional mean and variance estimates
        cond_mean = joint_mean
        cond_var = joint_var + epsilon  # Add epsilon for stability
        
        # Calculate Gaussian densities for conditional values
        cpds = torch.exp(-0.5 * ((given_values - cond_mean) ** 2) / cond_var) / torch.sqrt(2 * torch.pi * cond_var)
        return cpds


# Example usage
# Initialize estimator
estimator = ContinuousBayesianEstimator(prior_mean=0.5, prior_var=1.0, prior_dof=2.0, prior_scale=1.0)

# Update with observed data
observed_data = torch.tensor([2.0, 2.5, 3.0, 2.8])  # Example observed continuous data
estimator.update_posterior(observed_data)

# Sample from the posterior distribution
posterior_samples = estimator.sample_posterior(n_samples=5)
print("Posterior samples of the mean:", posterior_samples)

# Compute conditional probabilities for continuous given values
given_values = torch.tensor([2.5, 3.0, 3.5])  # Example given values
cpds = estimator.compute_cpds(observed_data, given_values)
print("Conditional Probability Densities (CPDs):", cpds)


Posterior samples of the mean: tensor([0.8179, 0.9911, 1.2523, 0.6728, 0.6119])
Conditional Probability Densities (CPDs): tensor([0.7603, 0.7742, 0.2212])
